In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/ubuntu/latent-diffusion-lm


In [2]:
from src.task.sgpt import SGPTTask
from coop.models.sgpt import SentenceGPT
import torch
from datasets import load_dataset
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra

/home/ubuntu/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()
initialize("../config/")
config = compose("sgpt")
config

/tmp/ipykernel_1045933/1463963186.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize("../config/")


{'optimizer': {'cls': 'adam', 'learning_rate': 0.001}, 'logger': {'name': 'wandb'}, 'dataset': {'train': ['heegyu/aihub_daily_conv_2022'], 'test': ['heegyu/aihub_daily_conv_2022']}, 'model': {'model': 'sgpt-mini.json', 'autoencoder': 'checkpoint/optimus-v2-44M/optimus-v2-tiny-vae.ckpt', 'latent_dim': 512, 'free_bit': 0.5, 'max_seq_len': 16, 'is_vae': False}, 'trainer': {'train_batch_size': 256, 'train_epochs': 3, 'shuffle': False, 'eval_batch_size': 256, 'eval_strategy': 'epoch', 'project': 'sgpt', 'run_name': 'base-opt56M', 'num_sanity_val_steps': 1}}

In [80]:
sgpt = SGPTTask(config).cuda()

In [81]:
batch = next(iter(sgpt.train_dataloader()))
batch[0]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using custom data configuration heegyu--aihub_daily_conv_2022-ee6ad77711466ab5
Reusing dataset parquet (/home/ubuntu/.cache/huggingface/datasets/heegyu___parquet/heegyu--aihub_daily_conv_2022-ee6ad77711466ab5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Using custom data configuration heegyu--aihub_daily_conv_2022-ee6ad77711466ab5
Reusing dataset parquet (/home/ubuntu/.cache/huggingface/datasets/heegyu___parquet/heegyu--aihub_daily_conv_2022-ee6ad77711466ab5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


{'id': 140329,
 'depth': 18,
 'speakers': [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2],
 'utterances': ['**는 게임 좋아하니?',
  '게임 어떤거 좋아해?',
  '게임 잘 못해서,',
  '우린 완전 게임 좋아해',
  '나는 별로 안 좋아해',
  '어떤 게임 주로 하세요?',
  '그럼... 보드 게임은 어대',
  '게임은 너무 시간낭비야',
  '보드게임은 단합하기 좋죵',
  '우리는 음 맞고도 치고 루미 큐브도 하고',
  '게임 할 시간에 난 책을 읽어',
  '요즘 보드게임 카페 잘 되어 있잖아용',
  '너 책 2년 동안 한권도 안 읽었...',
  '책을 읽는 게 좋아',
  '와우 엄청난 지식인',
  '보드 게임 카페가 있어?',
  '책은 정말 너무신기해',
  '넵 요즘 애들 다 보드게임 카페가용 키키'],
 'eous': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [82]:
loss, output = sgpt.step(batch, 0)

In [83]:
loss

tensor(62.3671, device='cuda:0', grad_fn=<DivBackward0>)

In [78]:
item = sgpt._item_for_train(batch[0]["utterances"][:4])
item

{'inputs': Normal(loc: torch.Size([16, 512]), scale: torch.Size([16, 512])),
 'attention_mask': tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 'labels': Normal(loc: torch.Size([16, 512]), scale: torch.Size([16, 512]))}

In [79]:
# item["inputs"].loc[1] - item["labels"].loc[0]
item["labels"]

Normal(loc: torch.Size([16, 512]), scale: torch.Size([16, 512]))

In [51]:
hidden = sgpt.model.model(
    inputs_embeds = item["inputs"].rsample().unsqueeze(0),
    attention_mask = item["attention_mask"].unsqueeze(0)
).last_hidden_state
x = sgpt.model.reparameterize(hidden)
hidden.shape, x

(torch.Size([1, 16, 512]),
 Normal(loc: torch.Size([1, 16, 512]), scale: torch.Size([1, 16, 512])))

In [68]:
from torch.distributions import Normal, kl_divergence

labels = Normal(
    item["labels"].loc.unsqueeze(0),
    item["labels"].scale.unsqueeze(0)
)
labels

Normal(loc: torch.Size([1, 16, 512]), scale: torch.Size([1, 16, 512]))

In [77]:
print(labels.loc[0, 2, :])

tensor([-1.0166e-01,  2.6448e-01,  4.6458e-01,  1.0069e-01, -1.2405e-01,
        -5.1134e-02, -1.4447e-01,  1.1437e-01,  5.2955e-01, -1.9023e-02,
         2.8022e-01,  3.5747e-01,  1.6090e-01, -4.1626e-02,  7.3284e-02,
        -1.9380e-01,  1.1013e+00,  9.2238e-02,  8.4387e-01,  1.7632e-01,
        -2.2174e-01, -4.0417e-01,  7.6656e-02,  3.6906e-01,  9.0171e-02,
        -1.3099e-01,  1.6960e-01, -5.4958e-01,  3.4345e-01,  1.7964e-01,
         5.0326e-01,  4.7890e-01, -2.0597e-02,  1.7637e-01, -2.6835e-01,
         1.7770e-01, -7.9549e-01, -1.0125e-01, -1.0733e-02,  9.0773e-02,
         1.6803e-01,  6.4719e-01,  3.9788e-01,  7.4002e-01,  6.0794e-01,
         3.9314e-01,  4.8075e-01, -4.4489e-01, -4.4656e-01,  5.4711e-01,
        -3.4176e-01, -7.0742e-02, -3.0671e-01, -2.6214e-01,  1.4209e-01,
        -2.7931e-01,  3.9125e-01, -4.0892e-01, -7.2583e-01, -3.9219e-01,
         3.6348e-02,  1.3070e-01,  2.7684e-01,  3.2558e-01, -2.7234e-01,
        -1.1607e-01,  2.2687e-04, -2.6895e-01, -3.6

In [72]:

def kldiv_loss(free_bit, p: Normal, q: Normal, attention_mask: torch.Tensor = None, use_free_bit: bool = True):
    zkl_real = kl_divergence(p, q).mean(-1)

    if attention_mask is not None:
        attn_sum = attention_mask.sum()
        zkl_real = zkl_real.masked_fill(attention_mask == 0, 0.0)
    else:
        attn_sum = p.loc.shape[0] * p.loc.shape[1]

    if use_free_bit:
        kl_mask = torch.gt(zkl_real, free_bit)
        zkl = zkl_real[kl_mask].sum() / attn_sum
        zkl_real = zkl_real.sum() / attn_sum

        return zkl, zkl_real
    else:
        zkl_real = zkl_real.sum() / attn_sum
        return zkl_real
        
# kldiv_loss(
#     0.5,
#     x,
#     labels,
#     attention_mask = item["attention_mask"],
#     use_free_bit=False
# )
kl_divergence(x, labels).mean(-1)

tensor([[5.2743e+01, 4.5951e+01, 5.6947e+01, 7.6523e+11, 7.5514e+11, 7.4267e+11,
         7.5617e+11, 7.6696e+11, 7.6777e+11, 7.3560e+11, 7.6166e+11, 7.3166e+11,
         7.4869e+11, 7.6578e+11, 7.6710e+11, 7.5890e+11]], device='cuda:0',
       grad_fn=<MeanBackward1>)